In [4]:
import sys
# sys.path.append(r'E:/0code')
sys.path.append(r'/home/wyf/0code')
# sys.path.append(r'/home/wangyf226/0code')
# sys.path.append(r'/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/pyml')

In [5]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import datetime
from pyml.emsemble.regression import GradientBoostingRegression
from pyml.feature_extraction.text import CountVectorizer
from pyml.linear_model.regression import LinearRegression
from pyml.neighbors.classification import KNeighborsClassifier
from pyml.metrics.regression import pearson_correlation
from pyml.model_selection import KFold
from pyml.model_selection import ShuffleSplit
from pyml.preprocessing import StandardScaler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'pyml.emsemble.regression'

# 读取数据文件

In [5]:
train = pd.read_excel('../data/train.xlsx')
test = pd.read_excel('../data/testStudent.xlsx')

In [6]:
train.dtypes # 检查有没有数据类型错误的，比如原本是int的变成str，说明里面可能有nan值等奇怪的数据

Additional_Number_of_Scoring                    int64
Average_Score                                 float64
Review_Total_Negative_Word_Counts               int64
Total_Number_of_Reviews                         int64
Review_Total_Positive_Word_Counts               int64
Total_Number_of_Reviews_Reviewer_Has_Given      int64
Tags                                           object
Reviewer_Score                                float64
dtype: object

In [7]:
train_ori_X = train.drop('Reviewer_Score', axis=1).drop('Tags', axis=1)
train_ori_Y = train['Reviewer_Score']
test_ori_X = test.drop('Tags', axis=1)

# 特征工程

In [8]:
def get_proportion_feature(df):
    """
    构造以下三个特征
    积极评论占总评论的比例
    消极评论占总评论的比例
    评论员评论占总评论的比例
    """
    df = df.copy()
    
    base_features = ['Total_Number_of_Reviews']
    gap_features = ['Review_Total_Negative_Word_Counts', 'Review_Total_Positive_Word_Counts','Total_Number_of_Reviews_Reviewer_Has_Given']
    for base_feature in base_features:
        for gap_feature in gap_features:
            df[gap_feature+'_radio_'+base_feature] = df[gap_feature]/df[base_feature]
            # 数字太小了，乘上一个10
#             df = df.drop(gap_feature, axis=1)
    return df

# 构造训练集和测试集，并归一化

In [9]:
train_X_feat = get_proportion_feature(train_ori_X)
test_X_feat = get_proportion_feature(test_ori_X)

In [10]:
train_X_feat.columns

Index(['Additional_Number_of_Scoring', 'Average_Score',
       'Review_Total_Negative_Word_Counts', 'Total_Number_of_Reviews',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given',
       'Review_Total_Negative_Word_Counts_radio_Total_Number_of_Reviews',
       'Review_Total_Positive_Word_Counts_radio_Total_Number_of_Reviews',
       'Total_Number_of_Reviews_Reviewer_Has_Given_radio_Total_Number_of_Reviews'],
      dtype='object')

In [11]:
ss = StandardScaler()
train_X = ss.fit_transform(train_X_feat.values)
test_X = ss.transform(test_X_feat.values)

In [12]:
train_Y = train_ori_Y.values

# 交叉验证

In [ ]:
n_splits = 3
cv = ShuffleSplit(n_splits=n_splits)
for train_indices, test_indices in cv.split(train_X):
    lr = GradientBoostingRegression(learning_rate=0.1, n_estimators=100, max_tree_node_size=400)
    lr.fit(train_X[train_indices], train_Y[train_indices], watch=True)
    y_pred = lr.predict(train_X[test_indices])
    print(pearson_correlation(y_pred, train_Y[test_indices]))

train 0/100  current cost : 254.82138866085717
train 1/100  current cost : 245.87921128422593
train 2/100  current cost : 237.80265193324854
train 3/100  current cost : 230.2228797284721
train 4/100  current cost : 222.98787005303743
train 5/100  current cost : 216.43380597810068
train 6/100  current cost : 210.07821114790872
train 7/100  current cost : 204.20736215640517
train 8/100  current cost : 198.33810388722918
train 9/100  current cost : 192.68409639682147
train 10/100  current cost : 187.04662918475574
train 11/100  current cost : 181.95729186993978
train 12/100  current cost : 177.2551589023348
train 13/100  current cost : 172.82128680783634
train 14/100  current cost : 168.7753691064605
train 15/100  current cost : 164.6137153531623
train 16/100  current cost : 160.62993147143882
train 17/100  current cost : 157.05362907115912
train 18/100  current cost : 153.71502384112884
train 19/100  current cost : 150.4817774527706
train 20/100  current cost : 147.33083714600463
train 2

# 训练模型写入结果

In [ ]:
lr = GradientBoostingRegression(learning_rate=0.1, n_estimators=100, max_tree_node_size=400)
lr.fit(train_X, train_Y, watch=True)

In [ ]:
y_pred = lr.predict(test_X)
sub = pd.DataFrame(y_pred)
sub.to_csv('../results/'+'GBDT-0.1-100-'+ str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + ".csv", index=0, header=None, index_label=None)